## Estimating Future Water Usage

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import numpy as np
import pandas as pd

def load_data_from_csv(
    window_size: int,
    csv_path: str = "../assets/plant_watering_data.csv",
    add_watering_flag: bool = True
    ):
    data = pd.read_csv(csv_path, parse_dates=["time", "watered_at"])
    data = data.sort_values("time")

    data["watered"] = data["pump_status"].astype(int)

    feature_cols = ["tank_level", "well_level"]
    if add_watering_flag:
    feature_cols.append("watered")

    features = data[feature_cols].values.astype("float32")

    mean = features.mean(axis=0)
    std = features.std(axis=0)
    features = (features - mean) / std

    X, y = [], []
    for i in range(len(features) - window_size):
    X.append(features[i:i + window_size])
    y.append(features[i + window_size, 0])

    return np.asarray(X), np.asarray(y), mean, std

window_size = 24
X, y, mean, std = load_data_from_csv(window_size)

model = Sequential([
    LSTM(64, input_shape=(window_size, X.shape[-1]), return_sequences=True),
    LSTM(32),
    Dense(1)
])

model.compile(optimizer="adam", loss="mse")
model.fit(X, y, epochs=50, batch_size=32, validation_split=0.2)
model.save("../models/water_model.h5")


I0000 00:00:1745717758.930045  116426 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 3487 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1050, pci bus id: 0000:01:00.0, compute capability: 6.1
/home/aries/projects/personal/LDC/LDC-Malin-LFirma/.venv/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
2025-04-27 02:35:59.736991: W external/local_xla/xla/service/gpu/llvm_gpu_backend/default/nvptx_libdevice_path.cc:40] Can't find libdevice directory ${CUDA_DIR}/nvvm/libdevice. This may result in compilation or runtime failures, if the program we try to run uses routines from libdevice.
Searched for CUDA in the following directories:
  ./cuda_sdk_lib
  ipykernel_launcher.runfiles/cuda_nvcc
  ipykern/cuda_nvcc
  
  /usr/local/cuda

InternalError: {{function_node __wrapped__FloorMod_device_/job:localhost/replica:0/task:0/device:GPU:0}} 'cuLaunchKernel(function, gridX, gridY, gridZ, blockX, blockY, blockZ, 0, reinterpret_cast<CUstream>(stream), params, nullptr)' failed with 'CUDA_ERROR_INVALID_HANDLE' [Op:FloorMod] name: 